<a href="https://colab.research.google.com/github/gurnoor6/opponent-modelling/blob/main/Opponent_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import shutil, os
filename_train = 'drive/MyDrive/rbc/trout_logs'
filename_validation = 'drive/MyDrive/rbc/trout_logs'
if os.path.exists(filename_train):
  shutil.rmtree(filename_train)
if os.path.exists(filename_validation):
  shutil.rmtree(filename_validation)
shutil.unpack_archive(f'{filename_train}.tar.gz', './')
filenames = os.listdir('./trout_logs')
print(len(filenames))
filenames.sort()

99189


In [4]:
train_num = int(0.95 * len(filenames))
train_filenames = filenames[:train_num]
val_filenames = filenames[train_num:]
print(train_filenames[0], val_filenames[0])

100008.npz 81246.npz


In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Num GPUs Available:  1
Found GPU at: /device:GPU:0


In [6]:
# Constants
T = 8 # Number of Timesteps for which boards are present in input
N = 8 # board size
HIDDEN_LAYER = 1 # hidden layer will be N * N * HIDDEN_LAYER neurons long
OUTPUT_LAYER = 73 # output layer will be N * N * OUTPUT_LAYER neurons long
BLOCKS = 10
FILTERS = 128

In [7]:
import numpy as np

class My_Custom_Generator(tf.keras.utils.Sequence) :
  
  def __init__(self, filenames, batch_size, randomize, folder_name) :
    self.filenames = filenames
    self.batch_size = batch_size
    self.randomize = randomize
    self.folder_name = folder_name
    self.boards_per_game = 4
    
    
  def __len__(self) :
    return (np.ceil(len(self.filenames) / float(self.batch_size))).astype(int)
  
  
  def __getitem__(self, idx) :
    filenames = self.filenames[idx * self.batch_size : (idx+1) * self.batch_size]

    inputs = []
    outputs = []
    legal_moves = []
    for f in filenames:
      if not f.endswith('.npz'): continue
      loaded = np.load(os.path.join(self.folder_name, f))
      length = len(loaded['inputs'])
      index = np.random.choice(length, length,replace=False)
      if self.randomize:
        index = np.random.choice(length, min(self.boards_per_game, length), replace = False)
      inputs.extend(loaded['inputs'][index])
      outputs.extend(loaded['outputs'][index])
      legal_moves.extend(loaded['legal_moves'][index])
    
    # Y_new = []
    # for item in outputs: Y_new.append(item.flatten())
    
    # print(np.array(inputs).shape, np.array(Y_new).shape)
    return (np.array(inputs), np.array(legal_moves)), np.array(outputs)

In [13]:
batch_size = 128

training_generator = My_Custom_Generator(train_filenames, batch_size, True, './trout_logs')
validation_generator = My_Custom_Generator(val_filenames, batch_size, False, './trout_logs')

In [14]:
inputs = tf.keras.Input(shape=(112, 8, 8))
legal_moves_input = tf.keras.Input(shape = (1858))
x1 = tf.keras.layers.LeakyReLU()(inputs)


x = tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", data_format = 'channels_first')(x1)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.LeakyReLU()(x)

x = tf.keras.layers.Conv2D(112, (3, 3), activation="relu", padding="same", data_format = 'channels_first')(x)
x = tf.keras.layers.BatchNormalization()(x)

added = tf.keras.layers.add([x1, x])
x = tf.keras.layers.LeakyReLU()(added)

x = tf.keras.layers.Conv2D(10, (3,3),activation="relu", padding="same", data_format = 'channels_first')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1858, activation = 'relu')(x)


# x = tf.keras.layers.add([x, legal_moves_input])

x = tf.keras.layers.Softmax()(x)
x = tf.keras.layers.Multiply()([x, legal_moves_input])
x = tf.keras.layers.Lambda(lambda t : t / tf.keras.backend.sum(t))(x)
model = tf.keras.Model(inputs = [inputs, legal_moves_input], outputs = x)
model.summary()
# tf.keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 112, 8, 8)]  0           []                               
                                                                                                  
 leaky_re_lu_4 (LeakyReLU)      (None, 112, 8, 8)    0           ['input_3[0][0]']                
                                                                                                  
 conv2d_3 (Conv2D)              (None, 128, 8, 8)    129152      ['leaky_re_lu_4[0][0]']          
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 128, 8, 8)   32          ['conv2d_3[0][0]']               
 rmalization)                                                                               

In [10]:
# inputs = tf.keras.Input(shape=(112, 8, 8))
# legal_moves_input = tf.keras.Input(shape = (1858))
# x1 = tf.keras.layers.LeakyReLU()(inputs)


# x = tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same", data_format = 'channels_first')(x1)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.LeakyReLU()(x)

# x = tf.keras.layers.Conv2D(112, (3, 3), activation="relu", padding="same", data_format = 'channels_first')(x)
# x = tf.keras.layers.BatchNormalization()(x)

# added = tf.keras.layers.add([x1, x])
# x = tf.keras.layers.LeakyReLU()(added)

# x = tf.keras.layers.Conv2D(10, (3,3),activation="relu", padding="same", data_format = 'channels_first')(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.LeakyReLU()(x)
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dense(1858, activation = 'relu')(x)


# # x = tf.keras.layers.add([x, legal_moves_input])

# x = tf.keras.layers.Softmax()(x)
# model = tf.keras.Model(inputs = [inputs, legal_moves_input], outputs = x)
# model.summary()
# # tf.keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    # loss=tf.keras.losses.MeanSquaredError(),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)

In [16]:
history = model.fit_generator(generator=training_generator,
                   steps_per_epoch = int(len(train_filenames) // batch_size),
                   epochs = 20,
                   verbose = 1,
                   validation_data = validation_generator,
                   validation_steps = int(len(val_filenames) // batch_size)  # Use only 1000 samples for validation
                  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/20
186/736 [======>.......................] - ETA: 17:46 - loss: 3.6787 - categorical_accuracy: 0.2921

KeyboardInterrupt: ignored

In [ ]:
# Plot the loss a sa function of number of epochs
import matplotlib.pyplot as plt
plt.title('Training Loss')
plt.xlabel('epochs')
plt.ylabel('Cross Entropy loss')
plt.plot(list(map(lambda x: x+1, range(len(history.history['loss'])))),  history.history['loss'])

In [ ]:
plt.title('Training Accuracy')
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.plot(list(map(lambda x: x+1, range(len(history.history['categorical_accuracy'])))),  history.history['categorical_accuracy'])

In [ ]:
# Plot the loss a sa function of number of epochs
plt.title('Validation Loss')
plt.xlabel('epochs')
plt.ylabel('Cross Entropy  loss')
plt.plot(list(map(lambda x: x+1, range(len(history.history['val_loss'])))),  history.history['val_loss'])

In [ ]:
# Plot the loss a sa function of number of epochs
plt.title('Validation Accuracy')
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.plot(list(map(lambda x: x+1, range(len(history.history['val_categorical_accuracy'])))),  history.history['val_categorical_accuracy'])

In [ ]:
print(history.history)

In [ ]:
from google.colab import files

folder_name = "saved-model"

if os.path.exists(folder_name):
  shutil.rmtree(folder_name)
os.mkdir(folder_name)

if os.path.exists(folder_name + ".zip"):
  os.remove(folder_name + ".zip")
model.save(folder_name)

shutil.make_archive(folder_name, 'zip', folder_name)
files.download(folder_name + ".zip")